In [77]:
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import os, os.path
import sys
sys.path.insert(0,'../processing/')
sys.path.insert(0,'../')
from datasets import sythtextprovider
from deployment import model_deploy
from nets import txtbox_300
#from processing import image_processing
from image_processing2 import *
from processing import ssd_vgg_preprocessing
import tf_utils
import time
slim = tf.contrib.slim

In [78]:
data_dir = '/Users/xiaodiu/Documents/github/projecttextbox/TextBoxes-TensorFlow/data/sythtext/'
file_name = data_dir + '1.tfrecord'
## test if file_name exists  

#example = tf.python_io.tf_record_iterator(file_name).next()
reader =  tf.python_io.tf_record_iterator(file_name)
#image_buffer, label, bboxes, name= parse_example(example)

In [85]:
def next_batch(reader,net,anchors,batch_size = 32):
    images = []
    b_glocal = []
    b_gsocres = []
    len = 0
    while(len < batch_size):
        try:
            example = reader.next()
            image_buffer, label, bboxes, name= parse_example(example)
            image = tf.image.decode_jpeg(image_buffer,channels=3)
            image, glabels, gbboxes,num =\
                ssd_vgg_preprocessing.preprocess_image(image,label,bboxes, 
                                                (300,300),is_training=True,
                                                data_format='NHWC')
            #glocalisations, gscores = \
            #net.bboxes_encode( gbboxes, anchors,num)
            #b_glocal.append(glocalisations)
            #b_gsocres.append(gscores)
            images.append(image)
            len +=1
        except StopIteration:
            print 'here is end ',i   
    return images, b_glocal, b_gsocres

In [86]:
with tf.Graph().as_default():
    ## build and test Nets
    net = txtbox_300.TextboxNet()
    image_shape = net.default_params.img_shape
    anchors = net.anchors(image_shape)
    
    images, b_glocal, b_gsocres = \
    next_batch(reader,net,anchors,1)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        tf.train.start_queue_runners(sess=sess)
        images = sess.run(images)
        
    

name SparseTensor(indices=Tensor("ParseSingleExample/Slice_Indices_image/name:0", shape=(?, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseExample/ParseExample:6", shape=(?,), dtype=string), dense_shape=Tensor("ParseSingleExample/Squeeze_Shape_image/name:0", shape=(1,), dtype=int64))


InvalidArgumentError: input boxes must be 3-dimensional [batch, num_boxes, coords]: [1,1,11,4]
	 [[Node: ssd_preprocessing_train/distorted_bounding_box_crop/SampleDistortedBoundingBox = SampleDistortedBoundingBox[T=DT_INT32, area_range=[0.1, 1], aspect_ratio_range=[0.8, 1.2], max_attempts=200, min_object_covered=0.05, seed=0, seed2=0, use_image_if_no_bounding_boxes=true, _device="/job:localhost/replica:0/task:0/cpu:0"](ssd_preprocessing_train/distorted_bounding_box_crop/Shape, ssd_preprocessing_train/distorted_bounding_box_crop/ExpandDims)]]

Caused by op u'ssd_preprocessing_train/distorted_bounding_box_crop/SampleDistortedBoundingBox', defined at:
  File "/Applications/python/anaconda/envs/tensorflow2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Applications/python/anaconda/envs/tensorflow2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Applications/python/anaconda/envs/tensorflow2

In [66]:
with tf.Session() as sess:

33